### import libraries

In [1]:
#for sql
!pip install sqldf
import sqldf
import pandas as pd
#for viz
import seaborn as sns
import matplotlib.pyplot as plt

# ***A History of F1: The Fastest Sport in the world***

Since its inception in 1950, Formula 1, often referred to as F1, has been celebrated as the pinnacle of motor racing. Emerging from a mix of amateur racing enthusiasts and automotive manufacturers, the sport has evolved and become renowned for being at the forefront of innovation and technology. It inspires millions of race fans worldwide as teams and drivers compete against each other and the clock, striving for the ultimate honor of being crowned world champions.

As a fan of F-1 since the late 90's, I'm going to dig into the data and see what insights I can gather and get a look at how the sport I love so dearly has evolved and also paint a picture for anyone curious to see what F1 is all about.

The dataset used spans the years 1950-1975 and can be found here:
[dataset](https://https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020?resource=download#)

## ***The Basics***

Let's start with the basics:

F-1 is comprised of teams who build the cars and employ drivers who race them. Although a team sport, it's usually the drivers who get all the attention. They are the most visible part of the team and in teams that are fielded by manufacturers, they also become the de-fato face of the brand. They also risk their lives since motor-racing is inherently a dangerous sport.

So, let's see what we can find out about the various teams and drivers who have a part of F1 in the past 75 years.

### ***Teams***

In [31]:
#grouping constructors and nationality
#read the csv file
constructors = pd.read_csv('/content/constructors.csv')

query="""

SELECT nationality,
COUNT(nationality) AS constructor_count,
RANK() OVER (ORDER BY COUNT(nationality) DESC) AS rank
FROM constructors
GROUP BY nationality
ORDER BY rank ASC
LIMIT 10;
"""
result = sqldf.run(query)
result

# #set figure size
# fig,ax=plt.subplots(figsize=(10,8))
# #set background color for the bar chart
# sns.set_style('darkgrid')
# #plot bar chart
# ax.grid(False)
# sns.barplot(data=result, x='nationality', y='constructor_count',ax=ax);
# #rotate the labels for visibility
# plt.xticks(rotation=90);

,nationality,constructor_count,rank
0,British,86,1
1,American,39,2
2,Italian,30,3
3,French,13,4
4,German,10,5
5,Swiss,5,6
6,Japanese,5,6
7,South African,3,8
8,Dutch,3,8
9,Russian,2,10


F1 is predominantly based in Europe with the UK dubbed as the Silicon Valley of the sport. Even most current teams have their racing HQ's and facilities in the UK. Hence, it is not unsurprising to see that most F1 teams are European and a majority overwhelmingly English.

### ***Type of Teams***

Delving into team histories is very convoluted because of the ways teams grow and evolve. F1 being very complex and expensive meant most teams did not last very long as they ran out of money. Teams also get bought-out or taken over by other wealthy individuals, consortiums, OEM's etc. and consequently get rebranded.


For example: the current Mercedes-AMG team traces it's lineage back to the 1980's when it was first owned and run by Ken Tyrrell before being bought by British American Tobacco who raced as British American Racing, then taken over by the Japanese behemoth Honda before being rescued from insolvency by famed F1 race engineer, Ross Brawn as Brawn GP in the wake of the 2008 economic crisis.

Independent teams also often tied-up with OEM's by way of sponsorship or exhange of technology. Such tie-ups led to rebranding of the team to reflect the new partnership. For example: Team Mclaren was rebranded to McLaren-Honda when they sourced their engines from Honda. They are now called McLaren-Mercedes as they now run Mercedes engines. This is also a way
team branding evolves.

Let's look at the composition of teams across the sports history:

In [32]:
#team composition

query="""
SELECT type AS type_of_constructor,
       count(*) AS total
FROM constructors
GROUP BY type
ORDER BY total DESC
"""
result=sqldf.run(query)
result

# #plot
# #set figure size
# fig,ax=plt.subplots(figsize=(5,5))
# #set background color for the bar chart
# sns.set_style('darkgrid')
# ax.grid(False)
# #plot bar chart
# sns.barplot(data=result, x='type_of_constructor', y='total');
# #rotate the labels for visibility
# # plt.xticks(rotation=90);

,type_of_constructor,total
0,Independent,157
1,Tie-Up,41
2,OEM,14


### ***Most successful Team of all time***

In [34]:
#combining results and races
#using DISTINCT to get the right number of races

results = pd.read_csv('/content/results.csv')
races = pd.read_csv('/content/races.csv')

query = """

SELECT
        constructors.constructorId,
        constructors.name,
        COUNT(DISTINCT(races.raceId)) AS total_races,
        COUNT(CASE WHEN results.positionOrder = 1 THEN 1 END) AS total_wins,
        ROUND(COUNT(CASE WHEN results.positionOrder = 1 THEN 1 END) * 100.0 / COUNT(DISTINCT(races.raceId)),2) AS win_percentage,
        COUNT(CASE WHEN results.positionOrder = 2 THEN 1 END) AS total_second_place_finishes,
        COUNT(CASE WHEN results.positionOrder = 3 THEN 1 END) AS total_third_place_finishes,
        COUNT(CASE WHEN results.positionOrder IN (2,3) THEN 1 END) AS total_podiums
FROM constructors
JOIN results ON
     constructors.constructorId = results.constructorId
JOIN races ON
     results.raceId = races.raceId
GROUP BY constructors.constructorId, constructors.name
ORDER BY total_wins DESC
LIMIT 10;

"""
#exequte query
wins2 = sqldf.run(query)
wins2

# #create pivot table for easy plotting
# wins2_pivot = wins2.pivot(index='name', columns='constructorId', values=['total_wins', 'win_percentage'])

# #plot the data
# fig,ax1=plt.subplots(figsize=(12,5))
# sns.set_style('darkgrid')
# #barplot for the wins
# sns.barplot(data=wins2, x='name', y='total_wins',ax=ax1,alpha=0.6,);

# ax1.set_title('Top 10 teams by Wins and Win Percentage')
# ax1.set_xlabel('') #leaving it blank for aesthetics
# ax1.set_ylabel('Total Wins')
# plt.xticks(rotation=0);

# #setting up the dual axis for win percentage
# ax2 = ax1.twinx()
# sns.lineplot(data=wins2, x='name', y='win_percentage',ax=ax2,color='green');
# ax2.set_ylabel('Win Percentages')

# #removing the grid for aesthetics
# ax1.grid(False)
# ax2.grid(False)

,constructorId,name,total_races,total_wins,win_percentage,total_second_place_finishes,total_third_place_finishes,total_podiums
0,6,Ferrari,1088,246,22.61,296,287,583
1,1,McLaren,917,180,19.63,168,148,316
2,131,Mercedes,305,127,41.64,97,69,166
3,9,Red Bull,382,120,31.41,81,76,157
4,3,Williams,831,114,13.72,113,87,200
5,32,Team Lotus,395,45,11.39,31,38,69
6,4,Renault,403,35,8.68,35,33,68
7,22,Benetton,260,27,10.38,32,43,75
8,25,Tyrrell,433,23,5.31,33,21,54
9,34,Brabham,308,23,7.47,25,30,55


There are two ways we can look at this:
1. total number of wins
2. number of wins based on appearances

The most succesful outright in F1 is the legendary Italian automaker, Ferrari with 246 race wins. But that also comes with a caveat: they are also the longest serving team in F1 i.e. they've been a part of the series since it's inception and so just by longevity in the sport, have the most wins

But if you were to judge success based on appearances i.e. a percentage, then Ferrari is the only 3rd most sucessful team in F1. Instead that crown falls to Mercedes from Germany.

In amongst the top 10 teams, are names like Bentton, Williams and Red Bull; names that are not usually associated with the automotive technology. This only serves to illustrate how complex the life of a team can be. For instance, Red Bull, a sports drink company first entered the sport as a sponsor in the 1990s before taking over the Jaguar team in 2005 and investing vast sums into establishing itself as one of the most successful teams in the sport.

### Constructors Champions


In [49]:
# query = """

# SELECT name,
#       COUNT(name) AS total_championships

# FROM
# (
# SELECT constructors.name,
#        races.year,
#        SUM(results.points) AS total_points,
#        RANK() OVER(PARTITION BY races.year ORDER BY SUM(results.points) DESC) AS rank
# FROM constructors
# JOIN results ON
#      constructors.constructorId = results.constructorId
# JOIN races ON
#      races.raceId = results.raceId
# GROUP BY constructors.name, races.year
# ORDER BY races.year DESC
# ) AS subquery

# WHERE subquery.rank=1
# GROUP BY name
# ORDER BY total_championships DESC

# """

# result = sqldf.run(query)
# result


# subquery not working

### Most successful Driver of all time

The topic of G.O.A.T like every other in every other sport is a contentious one. Unlike other sports based on individuals or teams of player, in F1 what you see on track is a only a part of the work that goes into racing. A large portion of success relies on what goes behind the scenes: building a good race car and the engineers who design and build them.



While there are clearly some drivers who are better than others, they also need to be paired with the best teams to translate their skills into racing success. Sadly, the sport is littered with many talented drivers not being able to succeed because of not being with the right team at the right time.

Nevertheless, let's look and see what we can glean from the data:

In [29]:
#load the required sheets: drivers and results
#results already loaded in the previous cell
drivers = pd.read_csv('/content/drivers.csv')

query = """
SELECT drivers.forename || " " || drivers.surname AS driver_name,
       COUNT(CASE WHEN results.positionOrder = 1 THEN 1 END) AS total_wins,
       COUNT(CASE WHEN results.positionOrder = 2 THEN 1 END) AS total_second_place,
       COUNT(CASE WHEN results.positionOrder = 3 THEN 1 END) AS total_third_place,
       COUNT(CASE WHEN results.positionOrder IN (2,3) THEN 1 END) AS total_podiums
FROM drivers
JOIN results ON
     drivers.driverId = results.driverId
GROUP BY driver_name
ORDER BY total_wins DESC
LIMIT 10;
"""
sqldf.run(query)

# #plot
# fig,ax=plt.subplots(figsize=(10,5))
# sns.set_style('darkgrid')
# ax.grid(False)
# sns.barplot(data=result, x='driver_name', y='total_wins',ax=ax);
# ax.set_title('Most Successful Drivers in F1');
# ax.set_xlabel(''); #leaving it blank for aesthetics
# ax.set_ylabel('Total Wins');
# ax.set_xticklabels(result['driver_name'],rotation=90);

,driver_name,total_wins,total_second_place,total_third_place,total_podiums
0,Lewis Hamilton,104,56,39,95
1,Michael Schumacher,91,43,21,64
2,Max Verstappen,61,30,16,46
3,Sebastian Vettel,53,36,33,69
4,Alain Prost,51,35,20,55
5,Ayrton Senna,41,23,16,39
6,Fernando Alonso,32,40,34,74
7,Nigel Mansell,31,17,11,28
8,Jackie Stewart,27,11,5,16
9,Niki Lauda,25,21,8,29


From the data, we can see that Sir Lewis Hamilton is the most successful driver in the sport.

### ***Driver Team Combination***



As I alluded to earlier, in F1, the car plays a big factor in the success of a driver. To use a soccer analogy: the teams with the best players are the most successful ones. They might not have the best player in every position, but overall, the quality of the team is higher. For that to happen, the team shouild be well-funded to be able to afford these players.

And it is a similar story in F1. The teams with the biggest budgets are able to invest more in R&D, hire the best engineers and drivers, have better infrastrcutre etc.

And that's also where we can take a deeper look at the data. Did Sir Lewis Hamilton win all his races with different teams under different circumstances or was it just with one dominant team?

To go back to my soccer analogy: Is Cristiano Ronaldo the G.O.A.T? He's had a trophy laden career for sure and all the statistcs make an easy case for that. But, hes won most of his titles at Real Madrid, the biggest club in the world, with the best team and coaches around him. Had he played for smaller clubs, then he most definetly wouldn't have one so many trophies. Case in point: Diego Maradona has never won the European Cup but is widely considered to be one of the G.O.A.T's if not the G.O.A.T




So, let's look at the most successful constructor-driver combimnation and see what that looks like:

In [36]:
query = """
SELECT drivers.forename || " " || drivers.surname AS driver_name,
       COUNT(CASE WHEN results.positionOrder = 1 THEN 1 END) AS total_wins,
       COUNT(CASE WHEN results.positionOrder = 2 THEN 1 END) AS total_second_place,
       COUNT(CASE WHEN results.positionOrder = 3 THEN 1 END) AS total_third_place,
       COUNT(CASE WHEN results.positionOrder IN (2,3) THEN 1 END) AS total_podiums,
       constructors.name AS constructor
FROM drivers
JOIN results ON
     drivers.driverId = results.driverId
JOIN constructors ON
      results.constructorId = constructors.constructorId
GROUP BY driver_name, constructor
ORDER BY total_wins DESC
LIMIT 10
"""
result = sqldf.run(query)
result


# # plot
# fig,ax = plt.subplots(figsize=(14,5))
# sns.set_style('darkgrid')
# ax.grid(False)
# sns.barplot(data=result, x='driver_name', y='total_wins', hue='constructor',ax=ax);
# ax.set_title('Diriver Team Combination');
# ax.set_xlabel(''); #leaving it blank for aesthetics
# ax.set_ylabel('Total Wins');
# ax.set_xticklabels(result['driver_name'],rotation=90);

# plot is not right

,driver_name,total_wins,total_second_place,total_third_place,total_podiums,constructor
0,Lewis Hamilton,83,40,27,67,Mercedes
1,Michael Schumacher,72,32,12,44,Ferrari
2,Max Verstappen,61,30,16,46,Red Bull
3,Sebastian Vettel,38,14,13,27,Red Bull
4,Ayrton Senna,35,12,8,20,McLaren
5,Alain Prost,30,21,12,33,McLaren
6,Nigel Mansell,28,12,3,15,Williams
7,Nico Rosberg,23,24,8,32,Mercedes
8,Damon Hill,21,14,5,19,Williams
9,Lewis Hamilton,21,16,12,28,McLaren


### ***Driver success by decade***

In [54]:
#number of wins >=10 by driver and the associated constructor
query = """

SELECT (FLOOR(strftime('%Y', date) / 10) * 10) || 's' AS decade,
       drivers.forename || " " || drivers.surname AS driver_name,
       constructors.name AS constructor,
       COUNT(CASE WHEN results.positionOrder = 1 THEN 1 END) AS total_wins
FROM races
JOIN results ON
     races.raceId = results.raceId
JOIN drivers ON
     results.driverId = drivers.driverId
JOIN constructors ON
     results.constructorId = constructors.constructorId
GROUP BY decade,driver_name,constructor
HAVING total_wins >= 10
ORDER BY decade ASC, total_wins DESC
"""
sqldf.run(query)

result = sqldf.run(query)
result

,decade,driver_name,constructor,total_wins
0,1950s,Alberto Ascari,Ferrari,13
1,1960s,Jim Clark,Lotus-Climax,19
2,1960s,Graham Hill,BRM,10
3,1970s,Jackie Stewart,Tyrrell,15
4,1970s,Niki Lauda,Ferrari,15
5,1970s,Mario Andretti,Team Lotus,11
6,1980s,Alain Prost,McLaren,30
7,1980s,Ayrton Senna,McLaren,14
8,1980s,Nelson Piquet,Brabham,13
9,1980s,Nigel Mansell,Williams,13
